# Import packages and CSV file

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('final_df.csv')

# Data Cleaning

Labeling the columns and getting rid of the extra index columns that remained after concatination of dataframes 

In [4]:
df = df.rename(columns={'0': 'mission', '1': 'tagline', '2': 'name', '3': 'category_name', '4': 'category_ID', '5': 'cause_name', '6': 'cause_ID', '7': 'city', '8': 'state', '9': 'zipcode', 
                   '10': 'current_score', '11': 'current_rating', '12': 'advisories', '13': 'accountability_score', '14': 'accountability_rating', '15': 'irs_org_type', '16': 'irs_classification', 
                   '17': 'npo_affilitation', '18': 'foundation_status', '19': 'irs_org_classification', '20': 'tax_deductibility', '21': 'npo_status', '22': 'org_assets', '23': 'org_income', 
                   '24': 'fin_score', '25': 'fin_rating'})

In [5]:
df['index'].unique

<bound method Series.unique of 0           0
1           1
2           2
3           3
4           4
         ... 
10813    8917
10814    8922
10815    8923
10816    8925
10817    8929
Name: index, Length: 10818, dtype: int64>

In [6]:
#Reviewing data types
df.dtypes

Unnamed: 0                  int64
index                       int64
mission                    object
tagline                    object
name                       object
category_name              object
category_ID                 int64
cause_name                 object
cause_ID                    int64
city                       object
state                      object
zipcode                     int64
current_score             float64
current_rating              int64
advisories                 object
accountability_score        int64
accountability_rating       int64
irs_org_type               object
irs_classification         object
npo_affilitation           object
foundation_status          object
irs_org_classification     object
tax_deductibility          object
npo_status                 object
org_assets                float64
org_income                float64
fin_score                 float64
fin_rating                  int64
dtype: object

In [7]:
#Dropping 'index' and 'Unnamed: 0'
df = df.drop(columns= ['index', 'Unnamed: 0'])

In [8]:
df.head()

,mission,tagline,name,category_name,category_ID,cause_name,cause_ID,city,state,zipcode,...,irs_classification,npo_affilitation,foundation_status,irs_org_classification,tax_deductibility,npo_status,org_assets,org_income,fin_score,fin_rating
0,Impact Alabama was founded in 2004 as the stat...,Engaging a Generation for Change.,Impact America,Human Services,6,Social Services,29,Birmingham,AL,35209,...,Educational Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,"Student Services, Organizations of Students",Contributions are deductible,501(c)(3),2032739.0,3364323.0,91.70,4
1,Four Corners Ministries exists to communicate ...,The Gospel Sent,Four Corners Ministries,Religion,9,Religious Activities,26,Opelika,AL,36801,...,Charitable Organization,Independent - the organization is an independe...,Organization that normally receives no more th...,Christian,Contributions are deductible,501(c)(3),2602296.0,1930948.0,96.10,4
2,The Mobile Symphony Orchestra enhances lives b...,Working to create an environment that allows f...,Mobile Symphony Orchestra,"Arts, Culture, Humanities",2,Performing Arts,4,Mobile,AL,36602,...,Educational Organization,Independent - the organization is an independe...,Organization that normally receives no more th...,Symphony Orchestras,Contributions are deductible,501(c)(3),1162334.0,1597054.0,66.45,1
3,Big Oak Ranch is a Christian home for children...,A Christian home for children needing a chance,Big Oak Ranch,Human Services,6,"Youth Development, Shelter, and Crisis Services",16,Springville,AL,35146,...,Charitable Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,Group Home (Long Term),Contributions are deductible,501(c)(3),49861768.0,22928887.0,99.11,4
4,The Community Foundation of Greater Huntsville...,A Smart Place to Give,Community Foundation of Greater Huntsville,Community Development,10,Community Foundations,22,Huntsville,AL,35801,...,Charitable Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,Community Foundations,Contributions are deductible,501(c)(3),25229948.0,12989927.0,95.00,4


In [9]:
df.shape

(10818, 26)

In [10]:
#check for null values
df.isnull().sum()

mission                       0
tagline                      23
name                          0
category_name                 0
category_ID                   0
cause_name                    0
cause_ID                      0
city                          0
state                         0
zipcode                       0
current_score                 0
current_rating                0
advisories                10794
accountability_score          0
accountability_rating         0
irs_org_type               1906
irs_classification           22
npo_affilitation             11
foundation_status             9
irs_org_classification     1906
tax_deductibility             9
npo_status                    9
org_assets                  117
org_income                  117
fin_score                     0
fin_rating                    0
dtype: int64

In [11]:
#Null values are tagline(23) advisories(10794), irs_org_type(1906), irs_classification(1906), tax_deductability(9), npo_status(9), org_assets(117), org_income(117)

In [12]:
#Drop advisories because there are 10794 missing values
df.drop(columns='advisories')

,mission,tagline,name,category_name,category_ID,cause_name,cause_ID,city,state,zipcode,...,irs_classification,npo_affilitation,foundation_status,irs_org_classification,tax_deductibility,npo_status,org_assets,org_income,fin_score,fin_rating
0,Impact Alabama was founded in 2004 as the stat...,Engaging a Generation for Change.,Impact America,Human Services,6,Social Services,29,Birmingham,AL,35209,...,Educational Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,"Student Services, Organizations of Students",Contributions are deductible,501(c)(3),2032739.0,3364323.0,91.70,4
1,Four Corners Ministries exists to communicate ...,The Gospel Sent,Four Corners Ministries,Religion,9,Religious Activities,26,Opelika,AL,36801,...,Charitable Organization,Independent - the organization is an independe...,Organization that normally receives no more th...,Christian,Contributions are deductible,501(c)(3),2602296.0,1930948.0,96.10,4
2,The Mobile Symphony Orchestra enhances lives b...,Working to create an environment that allows f...,Mobile Symphony Orchestra,"Arts, Culture, Humanities",2,Performing Arts,4,Mobile,AL,36602,...,Educational Organization,Independent - the organization is an independe...,Organization that normally receives no more th...,Symphony Orchestras,Contributions are deductible,501(c)(3),1162334.0,1597054.0,66.45,1
3,Big Oak Ranch is a Christian home for children...,A Christian home for children needing a chance,Big Oak Ranch,Human Services,6,"Youth Development, Shelter, and Crisis Services",16,Springville,AL,35146,...,Charitable Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,Group Home (Long Term),Contributions are deductible,501(c)(3),49861768.0,22928887.0,99.11,4
4,The Community Foundation of Greater Huntsville...,A Smart Place to Give,Community Foundation of Greater Huntsville,Community Development,10,Community Foundations,22,Huntsville,AL,35801,...,Charitable Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,Community Foundations,Contributions are deductible,501(c)(3),25229948.0,12989927.0,95.00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10813,The mission of the Blind Children's Learning C...,Destination Independence,Blind Children's Learning Center,Education,3,Special Education,38,Santa Ana,CA,92705,...,Charitable Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,NaN,Contributions are deductible,501(c)(3),2910090.0,3209288.0,92.50,4
10814,"VISION: ""To see a vibrant community of servant...",Changing the few who change the many,Asian Access,Religion,9,Religious Activities,26,Artesia,CA,90701,...,Educational Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,"Religion Related, Spiritual Development N.E.C.",Contributions are deductible,501(c)(3),2022340.0,2920052.0,83.45,3
10815,South Coast Repertory (SCR) was founded in 196...,Advancing the art of theatre in the service of...,South Coast Repertory,"Arts, Culture, Humanities",2,Performing Arts,4,Costa Mesa,CA,92626,...,Educational Organization,Independent - the organization is an independe...,School 170(b)(1)(A)(ii),Theater,Contributions are deductible,501(c)(3),69024438.0,27427101.0,88.72,3
10816,The Santa Barbara Rescue Mission (SBRM) is a m...,Rebuilding Broken Lives,Santa Barbara Rescue Mission,Human Services,6,Rescue Missions,44,Santa Barbara,CA,93103,...,Charitable Organization,Independent - the organization is an independe...,Organization which receives a substantial part...,"Homeless, Temporary Shelter For",Contributions are deductible,501(c)(3),12223443.0,3951340.0,77.11,2


In [43]:
#View category_name values
df['category_name'].value_counts()

Human Services                3029
Arts, Culture, Humanities     1565
Health                        1092
Community Development         1089
Education                      840
International                  754
Animals                        623
Religion                       559
Environment                    542
Human and Civil Rights         463
Research and Public Policy     262
Name: category_name, dtype: int64

In [42]:
#View cause_name values
df['cause_name'].value_counts()

Social Services                                              890
Youth Development, Shelter, and Crisis Services              726
Performing Arts                                              724
Museums                                                      514
United Ways                                                  507
Development and Relief Services                              476
Advocacy and Education                                       463
Children's and Family Services                               429
Youth Education Programs and Services                        429
Animal Rights, Welfare, and Services                         408
Environmental Protection and Conservation                    406
Religious Activities                                         377
Patient and Family Support                                   373
Housing and Neighborhood Development                         313
Diseases, Disorders, and Disciplines                         300
Food Banks, Food Pantries

In [49]:
#View current_score values


In [36]:
#View current ratings 
df['current_rating'].value_counts()

3    4955
4    4304
2    1238
1     272
0      49
Name: current_rating, dtype: int64

In [45]:
#View accountability_score distribution


In [38]:
#Accountability_rating value counts
df['accountability_rating'].value_counts()

4    6585
3    3183
2     632
1     322
0      96
Name: accountability_rating, dtype: int64

In [46]:
#irs_classification
df['irs_classification'].value_counts()

Charitable Organization                        8161
Educational Organization                       1926
Religious Organization                          367
Organization to Prevent Cruelty to Animals      231
Literary Organization                            43
Scientific Organization                          42
Organization to Prevent Cruelty to Children      21
Organization for Public Safety Testing            5
Name: irs_classification, dtype: int64

In [44]:
#irs_org_type
df['irs_org_type'].value_counts()

Arts, Culture and Humanities                             1472
Human Services - Multipurpose and Other                  1255
Philanthropy, Voluntarism and Grantmaking Foundations     837
International, Foreign Affairs and National Security      685
Animal-Related                                            554
Educational Institutions and Related Activities           502
Environmental Quality, Protection and Beautification      458
Religion-Related, Spiritual Development                   403
Youth Development                                         382
Health - General and Rehabilitative                       373
Housing, Shelter                                          290
Food, Agriculture and Nutrition                           289
Diseases, Disorders, Medical Disciplines                  195
Recreation, Sports, Leisure, Athletics                    179
Crime, Legal-Related                                      165
Civil Rights, Social Action, Advocacy                     158
Communit

In [39]:
#view npo_status
#the majority of the organizations are 501(c)(3)
df['npo_status'].value_counts()

501(c)(3)    10809
Name: npo_status, dtype: int64

In [40]:
#tax_deductibility 
df['tax_deductibility'].value_counts()

Contributions are deductible        10802
Deductibility not specified             6
Contributions are not deductible        1
Name: tax_deductibility, dtype: int64

In [ ]:
#org_assets and org_income

In [ ]:
#fin_score

In [47]:
#fin_rating
df['fin_rating'].value_counts()

3    4509
4    3883
2    1890
1     479
0      57
Name: fin_rating, dtype: int64

In [ ]:
#View NPO state distribution